In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
#from dataloader import SegmentationDataset
from model import UNet
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import csv
import time 
from datetime import datetime
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.nn.functional as F
import segmentation_models_pytorch as smp
import segmentation_models_pytorch.losses as smp_losses
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEGMENTATION_COLOURS = {0:[0,0,0],1:[255,0,0],2:[0,253,0],3:[0,0,250], 4:[253,255,0]}

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, num_classes, smooth=1e-6, ignore_index=None):
        super(DiceLoss, self).__init__()
        self.num_classes = num_classes
        self.smooth = smooth
        self.ignore_index = ignore_index

    def forward(self, logits, targets):
        

        probas = F.softmax(logits, dim=1)  

        # Convert targets to one-hot encoding
        targets_one_hot = F.one_hot(targets, num_classes=self.num_classes)
        targets_one_hot = targets_one_hot.permute(0, 3, 1, 2).float()  

        dims_to_sum = (0, 2, 3) 

        intersection = torch.sum(probas * targets_one_hot, dims_to_sum) 
        cardinality = torch.sum(probas + targets_one_hot, dims_to_sum)  

        dice_score_per_class = (2. * intersection + self.smooth) / (cardinality + self.smooth)  

        if self.ignore_index is not None:
            class_mask = torch.ones(self.num_classes, device=logits.device, dtype=torch.bool)
            if isinstance(self.ignore_index, int):
                class_mask[self.ignore_index] = False
            elif isinstance(self.ignore_index, (list, tuple)):
                for idx_to_ignore in self.ignore_index:
                    class_mask[idx_to_ignore] = False
            dice_score_per_class = dice_score_per_class[class_mask]
            if dice_score_per_class.numel() == 0: # All classes ignored
                return torch.tensor(0.0, device=logits.device, requires_grad=True)
        
        dice_loss = 1. - dice_score_per_class.mean()
        return dice_loss


In [4]:
def calculate_iou(pred_mask, target_mask, num_classes=5):
    ious = []
    
    if torch.is_tensor(pred_mask):
        pred_mask = pred_mask.cpu().numpy()
    if torch.is_tensor(target_mask):
        target_mask = target_mask.cpu().numpy()
    
    for cls in range(num_classes):
        pred_inds = pred_mask == cls
        target_inds = target_mask == cls
        
        intersection = np.logical_and(pred_inds, target_inds).sum()
        union = np.logical_or(pred_inds, target_inds).sum()
        
        iou = intersection / (union + 1e-6)  
        ious.append(iou)
    
    return np.mean(ious)

In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, root_dir, transform=None, target_size=(256, 256)):
        
        self.root_dir = root_dir
        self.transform = transform # This will be an albumentations pipeline
        self.target_size = target_size

        self.image_dir = os.path.join(root_dir, 'Images')
        self.mask_dir = os.path.join(root_dir, 'Labels')
        
        self.image_files = sorted([f for f in os.listdir(self.image_dir) 
                                 if f.endswith(('.png', '.jpg', '.jpeg'))])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_files[idx])
        base_name = os.path.splitext(self.image_files[idx])[0]
        mask_name = os.path.join(self.mask_dir, f"{base_name}_mask.png")
        
        # Load image and mask as PIL objects
        image_pil = Image.open(img_name).convert('RGB')
        mask_pil = Image.open(mask_name).convert('RGB')
        
        image_pil_resized = image_pil.resize(self.target_size, Image.LANCZOS) 
        mask_pil_resized = mask_pil.resize(self.target_size, Image.NEAREST)  

        # Convert resized PIL mask to NumPy array for color mapping
        mask_rgb_np = np.array(mask_pil_resized)
        class_mask_np = np.zeros((mask_rgb_np.shape[0], mask_rgb_np.shape[1]), dtype=np.uint8)
        
        # Map colors to class indices
        for class_idx, color in SEGMENTATION_COLOURS.items(): 
            matches = np.all(mask_rgb_np == np.array(color), axis=-1)
            class_mask_np[matches] = class_idx
        
        # Convert resized PIL image to NumPy array for albumentations
        image_np = np.array(image_pil_resized)
        
        if self.transform: # self.transform is the albumentations pipeline
            augmented = self.transform(image=image_np, mask=class_mask_np)
            image_tensor = augmented['image'] # ToTensorV2 handles conversion and channel order
            mask_tensor = augmented['mask'].long() # Mask should be LongTensor
        else:
            # Fallback if no albumentations transform is provided
            image_tensor = transforms.ToTensor()(image_np) # Handles HWC to CHW
            mask_tensor = torch.from_numpy(class_mask_np).long()
            
        return image_tensor, mask_tensor


In [6]:
# class SegmentationDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
        
#         self.root_dir = root_dir
#         self.transform = transform

#         self.image_dir = os.path.join(root_dir, 'Images')
#         self.mask_dir = os.path.join(root_dir, 'Labels')
        
#         self.image_files = sorted([f for f in os.listdir(self.image_dir) 
#                                  if f.endswith(('.png', '.jpg', '.jpeg'))])
        
#         for img in self.image_files:
#             base_name = os.path.splitext(img)[0]
#             mask_name = f"{base_name}_mask.png"
#             mask_path = os.path.join(self.mask_dir, mask_name)


#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         img_name = os.path.join(self.image_dir, self.image_files[idx])
#         base_name = os.path.splitext(self.image_files[idx])[0]
#         mask_name = os.path.join(self.mask_dir, f"{base_name}_mask.png")
        
#         # Load image and mask
#         image = Image.open(img_name).convert('RGB')
#         mask = Image.open(mask_name).convert('RGB')  # Keep as RGB for color mapping
        
#         if self.transform:
#             image = self.transform(image)
#             mask = self.transform_mask(mask)
        
#         # Convert RGB mask to class indices
#         mask_np = np.array(mask)
#         class_mask = np.zeros((mask_np.shape[0], mask_np.shape[1]), dtype=np.uint8)
        
#         # Map colors to class indices
#         for class_idx, color in SEGMENTATION_COLOURS.items():
#             # Find all pixels that match this color
#             matches = np.all(mask_np == np.array(color), axis=-1)
#             class_mask[matches] = class_idx
        
#         return image, torch.from_numpy(class_mask).long()
    
#     def transform_mask(self, mask):
#         """Apply only the spatial transforms to the mask (resize)"""
#         if self.transform is None:
#             return np.array(mask)
        
#         # Get the resize transform if it exists
#         for t in self.transform.transforms:
#             if isinstance(t, transforms.Resize):
#                 mask = t(mask)
#                 break
        
#         return np.array(mask)

In [7]:
def print_latest_metrics_table():
    metrics_dir = 'metrics'
    try:
        if not os.path.isdir(metrics_dir):
            print(f"Metrics directory '{metrics_dir}' not found. Run training first.")
            return

        all_files = [
            os.path.join(metrics_dir, f)
            for f in os.listdir(metrics_dir)
            if os.path.isfile(os.path.join(metrics_dir, f)) and
               f.startswith('training_metrics_') and f.endswith('.csv')
        ]
        if not all_files:
            print(f"No 'training_metrics_*.csv' files found in the '{metrics_dir}' directory.")
            return

        latest_file = max(all_files, key=os.path.getmtime)
        print(f"\n--- Training Metrics Summary from: {os.path.basename(latest_file)} ---")

        with open(latest_file, 'r', newline='') as f:
            reader = csv.reader(f)
            try:
                header = next(reader) # Read header row
                # Expected header: ['Epoch', 'Train Loss', 'Val Loss', 'Mean IoU', 'Time (s)']
                print(f"{'Epoch':<7} | {'Train Loss':<12} | {'Val Loss':<10} | {'Mean IoU':<10} | {'Time (s)':<10}")
                print("-" * 60) # Separator line

                for i, row in enumerate(reader):
                    if len(row) == 5:
                        try:
                            epoch = row[0]
                            train_loss = float(row[1])
                            val_loss = float(row[2])
                            mean_iou = float(row[3])
                            time_s = float(row[4])
                            print(f"{epoch:<7} | {train_loss:<12.4f} | {val_loss:<10.4f} | {mean_iou:<10.4f} | {time_s:<10.2f}")
                        except ValueError:
                            print(f"Warning: Could not parse row {i+1} in {os.path.basename(latest_file)}: {row}")
                    else:
                        print(f"Warning: Skipping malformed row {i+1} in {os.path.basename(latest_file)} (expected 5 columns): {row}")
            except StopIteration:
                print(f"Warning: The metrics file {os.path.basename(latest_file)} is empty or has no data rows.")

    except FileNotFoundError:
        print(f"Error: The directory '{metrics_dir}' or a specific metrics file was not found. Ensure training has run.")
    except Exception as e:
        print(f"An error occurred while trying to print metrics: {e}")

In [ ]:
def train_model(model, train_loader, val_loader, ce_criterion, dice_criterion, optimizer, num_epochs, device, loss_weights=(0.5, 0.5), scheduler=None):
    """
    Trains the model using a combined loss of CrossEntropy and Dice.
    ce_criterion: Instance of nn.CrossEntropyLoss
    dice_criterion: Instance of DiceLoss
    loss_weights: Tuple (ce_weight, dice_weight)
    scheduler: Optional learning rate scheduler
    """
    best_val_loss = float('inf')
    
    metrics_dir = 'metrics'
    os.makedirs(metrics_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    metrics_file = os.path.join(metrics_dir, f'training_metrics_{timestamp}.csv')
    
    with open(metrics_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Epoch', 'Train Loss', 'Val Loss', 'Mean IoU', 'Time (s)'])
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for images, masks in train_bar:
            images = images.to(device)
            masks = masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            
            loss_ce = ce_criterion(outputs, masks)
            loss_dice = dice_criterion(outputs, masks)
            loss = loss_weights[0] * loss_ce + loss_weights[1] * loss_dice
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_bar.set_postfix({'loss': train_loss / (train_bar.n + 1)}) # More accurate running loss
        
        avg_train_loss = train_loss / len(train_loader)
        
        model.eval()
        val_loss = 0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device)
                outputs = model(images)

                loss_ce_val = ce_criterion(outputs, masks)
                loss_dice_val = dice_criterion(outputs, masks)
                loss = loss_weights[0] * loss_ce_val + loss_weights[1] * loss_dice_val
                
                val_loss += loss.item()
                
                predictions = torch.argmax(outputs, dim=1)
                all_preds.append(predictions.cpu().numpy())
                all_targets.append(masks.cpu().numpy())
        
        avg_val_loss = val_loss / len(val_loader)
        
        all_preds = np.concatenate(all_preds, axis=0)
        all_targets = np.concatenate(all_targets, axis=0)
        
        # Determine num_classes for IoU calculation
        num_iou_classes = 5 # Default based on SEGMENTATION_COLOURS
        if hasattr(model, 'n_classes') and model.n_classes is not None: # For your custom UNet if it had this attr
            num_iou_classes = model.n_classes
        elif hasattr(model, 'segmentation_head') and hasattr(model.segmentation_head[0], 'out_channels'): # For SMP models
            num_iou_classes = model.segmentation_head[0].out_channels
        
        mean_iou = calculate_iou(all_preds, all_targets, num_classes=num_iou_classes)
        
        epoch_time = time.time() - epoch_start_time        
        with open(metrics_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([epoch+1, avg_train_loss, avg_val_loss, mean_iou, epoch_time])
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {avg_train_loss:.4f}')
        print(f'  Val Loss: {avg_val_loss:.4f}')
        print(f'  Mean IoU: {mean_iou:.4f}')
        print(f'  Time: {epoch_time:.2f}s')
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print('Model saved!')
        
        if scheduler:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(avg_val_loss) 
            else:
                scheduler.step() 


In [9]:
# def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
#     best_val_loss = float('inf')
    
#     metrics_dir = 'metrics'
#     os.makedirs(metrics_dir, exist_ok=True)
    
#     timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#     metrics_file = os.path.join(metrics_dir, f'training_metrics_{timestamp}.csv')
    
#     with open(metrics_file, 'w', newline='') as f:
#         writer = csv.writer(f)
#         writer.writerow(['Epoch', 'Train Loss', 'Val Loss', 'Mean IoU', 'Time (s)'])
    
#     for epoch in range(num_epochs):
#         epoch_start_time = time.time()
#         model.train()
#         train_loss = 0
#         train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
#         for images, masks in train_bar:
#             images = images.to(device)
#             masks = masks.to(device)
            
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, masks)
#             loss.backward()
#             optimizer.step()
            
#             train_loss += loss.item()
#             train_bar.set_postfix({'loss': train_loss / len(train_loader)})
        
#         avg_train_loss = train_loss / len(train_loader)
        
#         model.eval()
#         val_loss = 0
#         all_preds = []
#         all_targets = []
        
#         with torch.no_grad():
#             for images, masks in val_loader:
#                 images = images.to(device)
#                 masks = masks.to(device)
#                 outputs = model(images)
#                 loss = criterion(outputs, masks)
#                 val_loss += loss.item()
                
#                 predictions = torch.argmax(outputs, dim=1)
#                 all_preds.append(predictions.cpu().numpy())
#                 all_targets.append(masks.cpu().numpy())
        
#         avg_val_loss = val_loss / len(val_loader)
        
#         all_preds = np.concatenate(all_preds, axis=0)
#         all_targets = np.concatenate(all_targets, axis=0)
#         mean_iou = calculate_iou(all_preds, all_targets)
        
#         epoch_time = time.time() - epoch_start_time        
#         with open(metrics_file, 'a', newline='') as f:
#             writer = csv.writer(f)
#             writer.writerow([epoch+1, avg_train_loss, avg_val_loss, mean_iou, epoch_time])
        
#         print(f'Epoch {epoch+1}/{num_epochs}:')
#         print(f'  Train Loss: {avg_train_loss:.4f}')
#         print(f'  Val Loss: {avg_val_loss:.4f}')
#         print(f'  Mean IoU: {mean_iou:.4f}')
#         print(f'  Time: {epoch_time:.2f}s')
        
#         if avg_val_loss < best_val_loss:
#             best_val_loss = avg_val_loss
#             torch.save(model.state_dict(), 'best_model.pth')
#             print('Model saved!')


In [ ]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    target_image_size = (256, 256) 
    num_classes = 5 
    num_epochs_to_run = 120 

    # Albumentations transforms
    train_transform = A.Compose([
        A.Resize(height=target_image_size[0], width=target_image_size[1]),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.Rotate(limit=30, p=0.3),
        # A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
        # A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.3),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

    val_transform = A.Compose([
        A.Resize(height=target_image_size[0], width=target_image_size[1]),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

    train_dataset = SegmentationDataset('dataset/train', transform=train_transform, target_size=target_image_size)
    val_dataset = SegmentationDataset('dataset/val', transform=val_transform, target_size=target_image_size)

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)

    
    
    chosen_encoder = "efficientnet-b5" 
    print(f"Using model: DeepLabV3+ with {chosen_encoder} backbone")
    model = smp.DeepLabV3Plus(
        encoder_name=chosen_encoder,        
        encoder_weights="imagenet",     
        in_channels=3,                  
        classes=num_classes,            
    ).to(device)
    
    # Loss functions
    # ce_criterion = nn.CrossEntropyLoss().to(device)
    ce_criterion = smp_losses.FocalLoss(mode='multiclass').to(device)
    # Ensure DiceLoss class is defined in a previous cell
    dice_criterion = DiceLoss(num_classes=num_classes).to(device) 
    loss_combination_weights = (0.7, 0.3) 

    
    optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2) 

    # Learning Rate Scheduler
    # from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
    # scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=7, verbose=True, min_lr=1e-6)
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs_to_run, eta_min=1e-6) 

    train_model(model, train_loader, val_loader, 
                  ce_criterion, dice_criterion, 
                  optimizer, num_epochs=num_epochs_to_run, device=device, 
                  loss_weights=loss_combination_weights, scheduler=scheduler)
    
    # After training, print the metrics table
    print_latest_metrics_table()

if __name__ == '__main__':
    main()


Using device: cuda
Using model: DeepLabV3+ with efficientnet-b5 backbone


Epoch 1/120: 100%|██████████| 14/14 [00:07<00:00,  1.99it/s, loss=1.11]


Epoch 1/120:
  Train Loss: 1.1073
  Val Loss: 0.9896
  Mean IoU: 0.0044
  Time: 7.67s
Model saved!


Epoch 2/120: 100%|██████████| 14/14 [00:05<00:00,  2.41it/s, loss=1.01]


Epoch 2/120:
  Train Loss: 1.0058
  Val Loss: 1.0309
  Mean IoU: 0.0060
  Time: 6.35s


...

Epoch 118/120:
  Train Loss: 0.2574
  Val Loss: 0.2977
  Mean IoU: 0.3033
  Time: 6.36s


Epoch 119/120: 100%|██████████| 14/14 [00:05<00:00,  2.41it/s, loss=0.257]


Epoch 119/120:
  Train Loss: 0.2565
  Val Loss: 0.2983
  Mean IoU: 0.3029
  Time: 6.39s


Epoch 120/120: 100%|██████████| 14/14 [00:05<00:00,  2.43it/s, loss=0.256]


Epoch 120/120:
  Train Loss: 0.2557
  Val Loss: 0.2982
  Mean IoU: 0.3029
  Time: 6.34s

--- Training Metrics Summary from: training_metrics_20250507_103631.csv ---
Epoch   | Train Loss   | Val Loss   | Mean IoU   | Time (s)  
------------------------------------------------------------
1       | 1.1073       | 0.9896     | 0.0044     | 7.67      
2       | 1.0058       | 1.0309     | 0.0060     | 6.35      
3       | 0.9196       | 1.0113     | 0.0354     | 6.28      
4       | 0.8454       | 0.9425     | 0.0748     | 6.22      
5       | 0.7784       | 0.8509     | 0.1274     | 6.29      
6       | 0.7092       | 0.7766     | 0.1847     | 6.33      
7       | 0.6438       | 0.7383     | 0.2212     | 6.35      
8       | 0.5809       | 0.6827     | 0.2425     | 6.49      
9       | 0.5436       | 0.6610     | 0.2479     | 6.50      
10      | 0.5058       | 0.6371     | 0.2585     | 6.57      
11      | 0.4762       | 0.5876     | 0.2624     | 6.49      
12      | 0.4572       | 0.555